In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.11.6 (main, Oct  2 2023, 20:31:07) [Clang 16.0.3 ]


In [2]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [3]:
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)

API Key set: True


In [4]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from magnet.agents.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

✅ Agent imported successfully!


Let's create some example agent, to see how it works.

In [5]:
agent_example = Agent(
    name="Poet Agent",
    backstory="You are a well-known poet, who enjoys creating high quality poetry.",
    task_description="Write a poem about the meaning of life",
    task_expected_output="Just output the poem, without any title or introductory sentences",
)

In [6]:
print(agent_example.run())

In the cradle of dawn, where whispers reside,  
Life unfurls its tapestry, vast and wide.  
A dance of shadows, a play of light,  
In the heart of chaos, a quest for right.  

From the gentle sigh of the morning breeze,  
To the rustling leaves on ancient trees,  
Life's meaning hides in moments small,  
In the rise and fall, the call of all.  

In laughter shared and tears that fall,  
In the silent echoes of a lover's call,  
In dreams that soar on wings of night,  
In the courage found to face the fight.  

It's in the stories etched on wrinkled skin,  
In the battles lost, and those we win,  
In the fleeting glance of a stranger's eye,  
In the questions asked when we wonder why.  

Life's meaning, a puzzle, a mystery profound,  
In the spaces between, where silence is found.  
In love's embrace and time's gentle theft,  
In the heart's deep echo, the soul's quiet heft.  

So seek not far, nor high, nor wide,  
For life's true meaning dwells inside.  
In the simple breath, the pres

You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [7]:
from magnet.agents.tool_agent.tool import tool

In [8]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [9]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt, # type: ignore
)

In [10]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': './tool_agent_example.txt'}, 'id': 0}
Data successfully written to ./tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to './tool_agent_example.txt'."

## Defining Agent Dependencies

Let's define two agents now.

In [11]:
agent_1 = Agent(
    name="Poet Agent",
    backstory="You are a well-known poet, who enjoys creating high quality poetry.",
    task_description="Write a poem about the meaning of life",
    task_expected_output="Just output the poem, without any title or introductory sentences",
)

agent_2 = Agent(
    name="Poem Translator Agent",
    backstory="You are an expert translator especially skilled in Ancient Greek",
    task_description="Translate a poem into Ancient Greek", 
    task_expected_output="Just output the translated poem and nothing else"
)

We can define the agent dependencies using the `>>` operator.

In [12]:
agent_1 >> agent_2

Poem Translator Agent

This means `agent_2` depends on `agent_1`. We can check the dependencies and dependents of both agents.

In [13]:
print("Agent 1 dependencies: ", agent_1.dependencies)
print("Agent 1 dependents: ", agent_1.dependents)
print("Agent 2 dependencies: ", agent_2.dependencies)
print("Agent 2 dependents: ", agent_2.dependents)

Agent 1 dependencies:  []
Agent 1 dependents:  [Poem Translator Agent]
Agent 2 dependencies:  [Poet Agent]
Agent 2 dependents:  []


Now, if we run `agent_1`, the results will be added to `agent_2`'s context.

In [14]:
print(agent_1.run())

In the dance of dawn, where whispers weave,  
Life unfurls its tapestry, threads interleave.  
Moments like stars in the vast cosmic sea,  
Each a spark of wonder, wild and free.  

In laughter's echo and in sorrow's tear,  
Life finds its meaning, both far and near.  
In the gentle touch of a lover's hand,  
In dreams that drift like grains of sand.  

Through valleys deep and mountains high,  
In the silent gaze of the endless sky,  
Life's essence blooms in the heart's embrace,  
A journey of love, a timeless chase.  

In the quiet of night, when shadows play,  
Life whispers secrets in a soft ballet.  
In every heartbeat, in every breath,  
Life dances on, defying death.  

For meaning lies not in the end we seek,  
But in the moments that make us weak,  
In the courage to rise, to fall, to strive,  
In the simple truth that we are alive.  


In [15]:
print(agent_2.context)

Poem Translator Agent received context: 
In the dance of dawn, where whispers weave,  
Life unfurls its tapestry, threads interleave.  
Moments like stars in the vast cosmic sea,  
Each a spark of wonder, wild and free.  

In laughter's echo and in sorrow's tear,  
Life finds its meaning, both far and near.  
In the gentle touch of a lover's hand,  
In dreams that drift like grains of sand.  

Through valleys deep and mountains high,  
In the silent gaze of the endless sky,  
Life's essence blooms in the heart's embrace,  
A journey of love, a timeless chase.  

In the quiet of night, when shadows play,  
Life whispers secrets in a soft ballet.  
In every heartbeat, in every breath,  
Life dances on, defying death.  

For meaning lies not in the end we seek,  
But in the moments that make us weak,  
In the courage to rise, to fall, to strive,  
In the simple truth that we are alive.  


Now, if we run the second agent, it will use the context received from the previous agent to generate its output.

In [16]:
print(agent_2.run())

Ἐν τῇ ὀρχήστρᾳ τῆς ἕω, ὅπου ψιθυρισμοὶ ὑφαίνονται,  
Ἡ ζωή ἀναπτύσσει τὸν ἱστόν αὐτῆς, κλωστῆς διαπλέκονται.  
Στιγμαὶ ὡς ἄστρα ἐν τῇ ἀπείρῳ κοσμικῇ θαλάσσῃ,  
Ἕκαστον σπινθὴρ θαύματος, ἄγριον καὶ ἐλεύθερον.  

Ἐν τῇ ἠχῇ τοῦ γέλωτος καὶ ἐν τῷ δάκρυι τῆς λύπης,  
Ἡ ζωή εὑρίσκει τὸ νόημα αὐτῆς, καὶ μακρὰν καὶ ἐγγύς.  
Ἐν τῇ ἁπαλῇ ἀφῇ τῆς χειρὸς τοῦ ἐραστοῦ,  
Ἐν τοῖς ὀνείροις τοῖς πλανωμένοις ὡς κόκκοι ἄμμου.  

Διὰ τῶν κοιλάδων βαθέων καὶ τῶν ὀρέων ὑψηλῶν,  
Ἐν τῇ σιωπηλῇ ὄψει τοῦ ἀπείρου οὐρανοῦ,  
Ἡ οὐσία τῆς ζωῆς ἀνθεί ἐν τῇ ἀγκαλίᾳ τῆς καρδίας,  
Πορεία ἀγάπης, ἀέναος δίωξις.  

Ἐν τῇ ἡσυχίᾳ τῆς νυκτός, ὅταν σκιὰι παίζουσι,  
Ἡ ζωή ψιθυρίζει μυστικά ἐν ἀπαλῇ ὀρχήστρᾳ.  
Ἐν παντὶ κτύπῳ καρδίας, ἐν παντὶ πνοῇ,  
Ἡ ζωή ὀρχεῖται, θάνατον ἀψηφῶσα.  

Τὸ γὰρ νόημα οὐκ ἔνι ἐν τῷ τέλει ὃ ζητοῦμεν,  
Ἀλλ’ ἐν ταῖς στιγμαῖς αἳ ἡμᾶς ἀσθενεῖς ποιοῦσιν,  
Ἐν τῷ θάρρει ἀναστῆναι, πεσεῖν, ἐπιδιώκειν,  
Ἐν τῇ ἀπλῇ ἀληθείᾳ ὅτι ζῶμεν.  


## The Crew

In [17]:
from magnet.agents.multi_agent.crew import Crew

In [18]:
with Crew() as crew:
    agent_1 = Agent(
        name="Poet Agent",
        backstory="You are a well-known poet, who enjoys creating high quality poetry.",
        task_description="Write a poem about the meaning of life",
        task_expected_output="Just output the poem, without any title or introductory sentences",
    )

    agent_2 = Agent(
        name="Poem Translator Agent",
        backstory="You are an expert translator especially skilled in Spanish",
        task_description="Translate a poem into Spanish", 
        task_expected_output="Just output the translated poem and nothing else"
    )

    agent_3 = Agent(
        name="Writer Agent",
        backstory="You are an expert transcriber, that loves writing poems into txt files",
        task_description="You'll receive a Spanish poem in your context. You need to write the poem into './poem.txt' file",
        task_expected_output="A txt file containing the greek poem received from the context",
        tools=write_str_to_txt, # type: ignore
    )

    agent_1 >> agent_2 >> agent_3 # type: ignore

In [19]:
crew.run()


RUNNING AGENT: Poet Agent

In the cradle of dawn, where whispers reside,  
Life unfurls its tapestry, vast and wide.  
A dance of shadows and light's gentle grace,  
In every heartbeat, a story to trace.  

The river of time flows, relentless and true,  
Carving the valleys, painting skies blue.  
In moments of laughter and tears softly shed,  
The essence of being, in silence, is read.  

A tapestry woven with threads of the heart,  
Each soul a color, a work of fine art.  
In the quest for meaning, we wander and roam,  
Yet find in each other, a place to call home.  

The stars in their wisdom, they twinkle and gleam,  
Guiding our spirits through night’s tender dream.  
In the dance of existence, both fragile and bright,  
We find our purpose in love's gentle light.  

For life is a journey, a path to explore,  
With mysteries hidden behind every door.  
In the embrace of the earth and the sky,  
We seek and discover the reasons why.

RUNNING AGENT: Poem Translator Agent

En la cun